# 1. The difference in two population means 

## 1.1 Assumptions
- **Population variance is known. (Typically, we don't know the true population variance.)**

## 1.2 Formulas

$$SE(\bar{X_1} - \bar{X_2}) = \sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2} {n_2}} $$

$$Z = \frac{\bar{x_1}-\bar{x_2}}{SE}$$

In [1]:
# Function of performing the test
from scipy.stats import norm
import numpy as np

def mean_diff_two_p_Z(x1, x2, mudiff, sd1, sd2, n1, n2, side = 'two'):
    '''
    Outputs: Return z test-statistic and p-value 
    
    Inputs:
        x1: mean of the population 1
        x2: mean of the population 2
        mudiff: mean difference under the null hypothesis
        sd1: standard deviation of the population 1
        sd2: standard deviation of the population 2
        n1: sample size of the population 1
        n2: sample size of the population 2
        side: can be either two tail or one tail (default is two tail)
    '''
    
    # Hypothesis testing
    standard_error = np.sqrt(sd1**2/n1 + sd2**2/n2)
    z = ((x1 - x2) - mudiff)/standard_error
    
    # p-value given two tail or one tail
    if side == 'two':
        if z >= 0:
            pval = 2*(1-norm.cdf(z))
        elif z < 0:
            pval = 2*norm.cdf(z)
        
    elif side == 'less':
        pval = norm.cdf(z)
        
    elif side == 'larger':
        pval = 1 - norm.cdf(z)
    
    # Final outputs
    return round(z, 3), round(pval, 4)

## 1.3 Sampling Distribution
- **It follows a normal distribution regardless of the sample size, if sampling from normally distributed populations.**
$$\bar{X_1} - \bar{X_2} \sim N(\mu_1 - \mu_2, \frac{\sigma_1^2} {n_1} + \frac{\sigma_2^2} {n_2})$$

## 1.4 Question 
- **Male heights are approximately normally distributed with mean = 177.7 cm, and std = 5.6 cm. Female heights are approximately normally distributed with mean = 163 cm, and std = 5.1 cm. If 20 males and 15 females are randomly selected, what is the probability of the average height of the males is at least 10 cm greater than the average height of the females?**

$$H_0: \mu_1 - \mu_2 = 10$$

$$H_0: \mu_1 - \mu_2 \ge 10$$

In [2]:
# Followed by the description of the question, filled in the parameters
mean_diff_two_p_Z(x1 = 177.7, x2 = 163, mudiff = 10, 
                  sd1 = 5.6, sd2 = 5.1,
                  n1 = 20, n2 = 15,
                  side = 'larger')

(2.586, 0.0048)

# 2. The difference in two population means － Pooled-Variance T Tests

## 2.1 Assumptions
- Independent simple random samples.
- Normally distributed populations. (Or, as long as the sample size is large according to the CLT.)
- Equal population variances.

## 2.2 Formulas 

$$S_p^2 = \frac {(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2} {n_1 + n_2 - 2}$$

$$SE(\bar{X_1} - \bar{X_2}) = S_p \sqrt{\frac{1}{n_1} + \frac{1}{n_2}}$$

$$T = \frac{\bar{x_1}-\bar{x_2}}{SE}$$

$$d.f. = n_1+n_2-2$$

In [3]:
from scipy import stats
np.random.seed(0)

rvs1 = stats.norm.rvs(loc=9.76, scale=4.9, size=49)
rvs2 = stats.norm.rvs(loc=6.48, scale=3.49, size=70)

In [4]:
np.mean(rvs1), np.mean(rvs2)

(10.484070389586886, 6.866417734163768)

In [5]:
np.std(rvs1, ddof=1), np.std(rvs2, ddof = 1)

(5.623124706841221, 3.4442366887484837)

In [6]:
# P-value is computed as two-tails by default
stats.ttest_ind(rvs1, rvs2, equal_var=True)

Ttest_indResult(statistic=4.346423198139746, pvalue=2.9666761608266247e-05)

In [7]:
# Since the stats.ttest_ind requires two arrays as inputs, 
# I created a function doing tasks that don't require arrays as inputs
from scipy.stats import t
def mean_diff_two_p_t(x1, x2, mudiff, sd1, sd2, n1, n2, side = 'two'):
    '''
    Outputs: Return t test-statistic and p-value 
    
    Inputs:
        x1: mean of the population 1
        x2: mean of the population 2
        mudiff: mean under the null hypothesis
        sd1: standard deviation of the population 1
        sd2: standard deviation of the population 2
        n1: sample size of the population 1
        n2: sample size of the population 2
        side: can be either two tail or one tail 
    '''
    
    # Hypothesis testing
    pooled_variance = ((n1-1)*sd1**2 + (n2-1)*sd2**2) / (n1+n2-2)
    standard_error = np.sqrt(pooled_variance) * np.sqrt((1/n1) + (1/n2))
    t_stat = ((x1 - x2) - mudiff)/standard_error
    
    # p-value computation given two tails or one tail
    if side == 'two':
        if t_stat >= 0:
            pval = 2*(1-t.cdf(x = t_stat, df = n1+n2-2))
        elif t_stat < 0:
            pval = 2*t.cdf(x = t_stat, df = n1+n2-2)
        
    elif side == 'less':
        pval = t.cdf(x = t_stat, df = n1+n2-2)
        
    elif side == 'larger':
        pval = 1 - t.cdf(x = t_stat, df = n1+n2-2)
    
    # Final outputs
    return round(t_stat, 3), pval

In [8]:
# Compare the results between using the function and the scipy package
mean_diff_two_p_t(x1 = 10.484070389586886, x2 = 6.866417734163768, mudiff = 0, 
                  sd1 = 5.623124706841221, sd2 = 3.4442366887484837, 
                  n1 = 49, n2 = 70,
                 side = 'two')

(4.346, 2.9666761608204695e-05)

## 2.3 Question
- **The Morel Emotional Numbing Test (MENT) was designed as a diagnostic tool to help differentiate between those who truly had PTSD and those who were malingering. 49 from the group of seeking compensation has the mean score = 9.76 and std = 4.9. 70 from the group of seeking treatment has the mean score = 6.48 and std = 3.49.**

- **Test the null hypothesis that the populations have equal mean scores on the MENT.**

$$H_0: \mu_1 = \mu_2$$

$$H_a: \mu_1 \neq \mu_2$$

In [9]:
# The p-value is pretty small so that we should reject the null hypothesis that 
# the two populations have equal mean scores on the MENT.
mean_diff_two_p_t(x1 = 9.76, x2 = 6.48, mudiff = 0, 
                  sd1 = 4.9, sd2 = 3.49, 
                  n1 = 49, n2 = 70,
                 side = 'two')

(4.267, 4.047130771600571e-05)

- **Construct a 95% confidence interval for the population difference (mu1 - mu2).**

$$(\bar{x_1} - \bar{x_2}) \pm t_\frac{\alpha}{2} * SE(\bar{x_1}-\bar{x_2})$$

In [10]:
def mean_diff_two_p_t(x1, x2, mudiff, sd1, sd2, n1, n2, confidence_alpha, side = 'two'):
    '''
    Outputs: Return t test-statistic, p-value, and confidence intervals of the estimator
    
    Inputs:
        x1: mean of the population 1
        x2: mean of the population 2
        mudiff: mean under the null hypothesis
        sd1: standard deviation of the population 1
        sd2: standard deviation of the population 2
        n1: sample size of the population 1
        n2: sample size of the population 2
        confidence_alpha: usually 0.95
        side: can be either two tail or one tail
    '''
    
    # Hypothesis testing
    pooled_variance = ((n1-1)*sd1**2 + (n2-1)*sd2**2) / (n1+n2-2)
    standard_error = np.sqrt(pooled_variance) * np.sqrt((1/n1) + (1/n2))
    t_stat = ((x1 - x2) - mudiff)/standard_error
    
    # p-value computation given two tails or one tail
    if side == 'two':
        if t_stat >= 0:
            pval = 2*(1-t.cdf(x = t_stat, df = n1+n2-2))
        elif t_stat < 0:
            pval = 2*t.cdf(x = t_stat, df = n1+n2-2)
        
    elif side == 'less':
        pval = t.cdf(x = t_stat, df = n1+n2-2)
        
    elif side == 'larger':
        pval = 1 - t.cdf(x = t_stat, df = n1+n2-2)
    
    # Confidence intervals
    population_difference = x1 - x2 
    t_quantile = t.ppf(q = (1 - confidence_alpha)/2, df = n1+n2-2)
    margin_error = t_quantile*standard_error
    upper_confidence = population_difference + margin_error
    lower_confidence = population_difference - margin_error
    
    # Final outputs
    return round(t_stat, 3), pval, (upper_confidence, lower_confidence)

In [11]:
mean_diff_two_p_t(x1 = 9.76, x2 = 6.48, mudiff = 0, 
                  sd1 = 4.9, sd2 = 3.49, 
                  n1 = 49, n2 = 70,
                 side = 'two', confidence_alpha = 0.95)

(4.267, 4.047130771600571e-05, (1.7575594214519823, 4.802440578548016))

# 3. The difference in two population means － Unpooled-Variance T Tests

## 3.1 Assumptions
- Independent simple random samples.
- Normally distributed populations. (Or, as long as the sample size is large according to the CLT.)
- Do not assume equal variances.

## 3.2 Formulas

$$SE(\bar{X_1} - \bar{X_2}) = \sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2} {n_2}} $$

$$T = \frac{\bar{x_1}-\bar{x_2}}{SE}$$

$$d.f. = \frac {(\frac{s_1^2}{n_1} + \frac{s_2^2} {n_2})^2} 
{ \frac {1}{n_1-1} (\frac {s_1^2}{n_1})^2 + \frac {1} {n_2-1} (\frac {s_2^2}{n_2})^2 }$$

$$T = \frac{\bar{x_1}-\bar{x_2}}{SE}$$

## 3.3 Question 
- **Does a certain rattlesnake antivenom given after a rattlesnake bite have an effect on swelling? Researchers ran an experiment to investigate this problem by randomly assigning pigs to one of two groups. A treatment group, which received an antivenom. A placebo group, which received a saline solution. Changes in leg volume after 8 hours of receiving either an antivenom or a saline solution has been measured and recorded.**

- **Antivenom group has the mean = 203.33, std = 56.18, and sample size = 9. Saline group has the mean = 201.25, std = 112.62, and sample size = 8.**

- **(1) Test the null hypothesis that the populations have equal mean change in leg volume.**

$$H_0: \mu_1 = \mu_2$$

$$H_a: \mu_1 \neq \mu_2$$

- **(2) Construct a 95% confidence interval for the population difference (mu1 - mu2).**

In [12]:
# Final version of the mean_diff_two_p_t function
def mean_diff_two_p_t(x1, x2, mudiff, sd1, sd2, n1, n2, equal_var, confidence_alpha, side = 'two'):
    '''
    Outputs: Return t test-statistic, p-value, and confidence intervals of the estimator
    
    Inputs:
        x1: mean of the population 1
        x2: mean of the population 2
        mudiff: mean under the null hypothesis
        sd1: standard deviation of the population 1
        sd2: standard deviation of the population 2
        n1: sample size of the population 1
        n2: sample size of the population 2
        equal_var: equal varaince assumption (either False or True)
        confidence_alpha: usually 0.95
        side: can be either two tail or one tail
    '''
    
    # Hypothesis testing (unequal VS equal variance)
    if equal_var == True:
        pooled_variance = ((n1-1)*sd1**2 + (n2-1)*sd2**2) / (n1+n2-2)
        standard_error = np.sqrt(pooled_variance) * np.sqrt((1/n1) + (1/n2))
        t_stat = ((x1 - x2) - mudiff)/standard_error
        equal_df = n1+n2-2
        
    elif equal_var == False:
        standard_error = np.sqrt((sd1**2 / n1) + (sd2**2 / n2))
        t_stat = ((x1 - x2) - mudiff)/standard_error
        unequal_df = (( (sd1**2 / n1) + (sd2**2 / n2) ) ** 2) / ( (1/(n1-1))*(((sd1**2)/n1)**2) + (1/(n2-1))*(((sd2**2)/n2)**2) ) 
    
    # p-value (equal variance case)
    if (equal_var == True) and (side == 'two'):
        if t_stat >= 0:
            pval = 2*(1-t.cdf(x = t_stat, df = equal_df))
        elif t_stat < 0:
            pval = 2*t.cdf(x = t_stat, df = equal_df)
        
    elif (equal_var == True) and (side == 'less'):
        pval = t.cdf(x = t_stat, df = equal_df)
        
    elif (equal_var == True) and (side == 'larger'):
        pval = 1 - t.cdf(x = t_stat, df = equal_df)
        
    # p-value (unequal variance case)
    if (equal_var == False) and (side == 'two'):
        if t_stat >= 0:
            pval = 2*(1-t.cdf(x = t_stat, df = unequal_df))
        elif t_stat < 0:
            pval = 2*t.cdf(x = t_stat, df = unequal_df)
        
    elif (equal_var == True) and (side == 'less'):
        pval = t.cdf(x = t_stat, df = unequal_df)
        
    elif (equal_var == True) and (side == 'larger'):
        pval = 1 - t.cdf(x = t_stat, df = unequal_df)
    

    # Confidence intervals
    population_difference = x1 - x2
    
    if equal_var == True:
        t_quantile = t.ppf(q = (1 - confidence_alpha)/2, df = equal_df)
    elif equal_var == False:
        t_quantile = t.ppf(q = (1 - confidence_alpha)/2, df = unequal_df)

    margin_error = t_quantile*standard_error
    upper_confidence = population_difference + margin_error
    lower_confidence = population_difference - margin_error
    
    # Final outputs
    return round(t_stat, 3), pval, (upper_confidence, lower_confidence)

In [13]:
# p-value is very high so that we should not reject the null hypothesis that
# the populations have equal mean change in leg volume.
mean_diff_two_p_t(
    x1 = 203.33, x2 = 201.25, mudiff = 0, 
    sd1 = 56.18, sd2 = 112.62, 
    n1 = 9, n2 = 8,
    equal_var = False, side = 'two', confidence_alpha = 0.95
)

(0.047, 0.9632265708261047, (-95.94636525583655, 100.10636525583658))